In [ ]:
IMG_ROWS = 256
IMG_COLS = 256

In [ ]:
import cv2
import numpy as np
import imageio
from scipy import ndimage
from glob import glob
import zipfile
import tensorflow as tf
!pip3 install -U segmentation-models
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import tensorflow as tf
tf.keras.backend.set_image_data_format('channels_last')

In [ ]:
train_img_paths = sorted(glob('..E:/cell_bi2/aug_images_bi/*.JPG'))
train_mask_paths = sorted(glob('..E:/cell_bi2/aug_masks_bi/*.png'))

test_img_paths = sorted(glob('..E:/Inclusive-f/image/*.jpg'))
test_mask_paths = sorted(glob('..E:/Inclusive-f/Mask/*.png'))

In [ ]:
train_imgs = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))
                        for path in train_img_paths])

train_masks = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))
                        for path in train_mask_paths])

train_masks = train_masks.astype(np.float32)
train_masks[train_masks<=127] = 0.
train_masks[train_masks>127] = 1.
train_masks = np.reshape(train_masks, (*train_masks.shape, 1))

In [ ]:
test_imgs = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))
                        for path in test_img_paths])

test_masks = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))
                        for path in test_mask_paths])

test_masks = test_masks.astype(np.float32)
test_masks[test_masks<=127] = 0.
test_masks[test_masks>127] = 1.
test_masks = np.reshape(test_masks, (*test_masks.shape, 1))

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(0, figsize=(200, 200))
fig.add_subplot(1, 2, 1)
plt.imshow(train_imgs[0])
fig.add_subplot(1, 2, 2)
plt.imshow(np.squeeze(train_masks[0]), cmap='gray')

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(0, figsize=(20, 20))
fig.add_subplot(1, 2, 1)
plt.imshow(test_imgs[0])
fig.add_subplot(1, 2, 2)
plt.imshow(np.squeeze(test_masks[0]), cmap='gray')

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import segmentation_models as sm  

sm.set_framework('tf.keras')
sm.framework()

optimizer = Adam(learning_rate=0.001)
dice_loss = sm.losses.DiceLoss()

def build_unet(input_shape=(256, 256, 3)):
    inputs = layers.Input(shape=input_shape)

    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, 3, activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, 3, activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, 3, activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, 3, activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, 3, activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, 3, activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, 3, activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, 3, activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

model = build_unet()
model.compile(optimizer=optimizer, loss=dice_loss, metrics=['accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=500, restore_best_weights=True)

history = model.fit(
    train_imgs[10:],
    train_masks[10:],
    batch_size=4,
    epochs=200,
    validation_data=(
        train_imgs[10:],
        train_masks[10:]
    ),
    callbacks=[early_stopping]
)

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

save_dir = "pred_masks1"
os.makedirs(save_dir, exist_ok=True)

test_index = 0
test_image = test_imgs[test_index]
ground_truth_mask = test_masks[test_index].squeeze()
predicted_mask = model.predict(np.expand_dims(test_image, axis=0))[0]
pred_mask_uint8 = (predicted_mask * 255).astype(np.uint8)

save_path = os.path.join(save_dir, f"pred_mask_{test_index}.png")
cv2.imwrite(save_path, pred_mask_uint8)

print("Saved predicted mask:", save_path)

plt.figure(figsize=(10, 10))

plt.subplot(131)
plt.imshow(test_image)
plt.title('Test Image')
plt.axis('off')

plt.subplot(132)
plt.imshow(ground_truth_mask, cmap='gray')
plt.title('Ground Truth Mask')
plt.axis('off')

plt.subplot(133)
plt.imshow(predicted_mask, cmap='gray')
plt.title('Predicted Mask')
plt.axis('off')

plt.tight_layout()
plt.savefig("comparison_output.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import save_model

save_model(model, 'E:/segmentation_model_unetCELLS.h5')
print("Model saved as 'segmentation_model.h5'")

In [ ]:
test_loss, test_accuracy = model.evaluate(test_imgs, test_masks, batch_size=1)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)